In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import argparse
import os
import plotly.graph_objects as go
import plotly.io as pio
from time import sleep

# Parameters

In [2]:
box_size=50.0
r=0.5
dataCompression=500.0
numSteps=100000.0
timestep=0.001

# Load Data

In [3]:
# Load the JSON file
with open('particle_positions.json', 'r') as file:
    data = json.load(file)

# Convert the data to a NumPy array
particle_positions = np.array(data)

### Plot Energy

In [4]:
def energy_plotly_plotter(xlabel, ylabel, legend_names, colors):
    # Read data from the file
    
    # Load the JSON file
    with open('particle_positions.json', 'r') as file:
        data = json.load(file)

    # Convert the data to a NumPy array
    particle_positions = np.array(data)

    time = []
    kinetic_energy = []
    potential_energy = []
    total_energy = []

    for t in range(len(particle_positions)):
        time.append(float(t)*dataCompression*timestep)
        kinetic_energy_total=0.0
        i=0
        for x, y ,vx ,vy , phi, PE in particle_positions[t]:
            kinetic_energy_total+=(float(0.5 * (vx**2 + vy**2)))
            if i==len(particle_positions[0])-1:
                potential_energy_per_particle=float(PE)
                kinetic_energy_per_particle=kinetic_energy_total/float(len(particle_positions[0]))
            i+=1
        potential_energy.append(potential_energy_per_particle)
        kinetic_energy.append(kinetic_energy_per_particle)
        total_energy.append(potential_energy_per_particle+kinetic_energy_per_particle)

    # Create Plotly figure
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=time, y=kinetic_energy, mode='lines', name=legend_names[0], line=dict(color=colors[0])))
    fig.add_trace(go.Scatter(x=time, y=potential_energy, mode='lines', name=legend_names[1], line=dict(color=colors[1])))
    fig.add_trace(go.Scatter(x=time, y=total_energy, mode='lines', name=legend_names[2], line=dict(color=colors[2])))

    fig.update_layout(title="Energy Plot", xaxis_title=xlabel, yaxis_title=ylabel, width=1200, height=600)

    # Save or show the plot
    # pio.write_image(fig, 'tempi.png')
    pio.write_html(fig, 'tempi.html')


In [ ]:
# Save the plot as PNG
f=0
legend_names = ["Kinetic Energy", "Potential Energy", "Total Energy"]
colors = ['blue', 'green', 'red']
while True:
        energy_plotly_plotter("Time", "Energy per Particle", legend_names, colors)
        if f<1:
            os.system(f"start {os.getcwd()}/tempi.html")
        f+=1
        sleep(10) #change to increase delay

### Final Frame Plotter

In [ ]:
# Create a figure and axis for the Plot
fig, ax = plt.subplots()
# Set the axis limits based on your particle box size
ax.set_xlim(0, box_size)
ax.set_ylim(0, box_size)

def FinalSysImagePlotter():
    ax.clear()
    ax.set_xlim(0, box_size)
    ax.set_ylim(0, box_size)
    
    for x, y ,vx ,vy , phi, PE in particle_positions[-1]:
        circle = plt.Circle((x, y), radius=r, linewidth=0)
        ax.add_patch(circle)
    
    # Add an arrow to represent the direction of movement
        arrow = plt.Arrow(x, y, vx, vy, width=0.1, color='red')
        ax.add_patch(arrow)
        
    # Add an arrow to represent the direction of movement
        arrow = plt.Arrow(x, y, np.cos(phi), np.sin(phi), width=0.1, color='green')
        ax.add_patch(arrow)
    
    # Set axis labels and title
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.title('Final Image of System')
    
    # Show the plot
    plt.show()
    plt.savefig("FinalFrame.png")

#Plots the final system image
FinalSysImagePlotter()

### Animation

In [ ]:
# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Set the axis limits based on your particle box size
ax.set_xlim(0, box_size)
ax.set_ylim(0, box_size)

# Function to update the animation
def update(frame):
    ax.clear()
    ax.set_xlim(0, box_size)
    ax.set_ylim(0, box_size)

    # Plot the particles at the given frame as circles
    for x, y , vx, vy, phi, PE in particle_positions[frame]:
        circle = plt.Circle((x, y), radius=r, linewidth=0)
        ax.add_patch(circle)
        
    # Add an arrow to represent the direction of movement
        arrow = plt.Arrow(x, y, vx, vy, width=0.1, color='red')
        ax.add_patch(arrow)

    # Add an arrow to represent the direction of movement
        arrow = plt.Arrow(x, y, np.cos(phi), np.sin(phi), width=0.1, color='green')
        ax.add_patch(arrow)

# Create the animation
animation = FuncAnimation(fig, update, frames=len(particle_positions), interval=100)

# Save the animation as a GIF
# animation.save('particle_animation.gif', writer='pillow')  # For GIF
# Save the animation as a video
animation.save('particle_animation.mp4', writer='ffmpeg')  # For MP4


### Vis Py

In [ ]:
import numpy as np
from vispy import scene, app

# Create a canvas
canvas = scene.SceneCanvas(keys='interactive', bgcolor='white', show=True)

# Create a view
view = canvas.central_widget.add_view()

# Set the view limits based on your particle box size
box_size = 10  # Adjust this to match your particle box size
view.camera.set_range(x=(0, box_size), y=(0, box_size))

# Create a scatter plot to represent particles
particles = scene.Markers()
view.add(particles)

# Function to update the animation
def update(frame):
    positions = particle_positions[frame]  # Replace with your particle data for the frame
    
    # Update the particle positions
    particles.set_data(positions[:, :2], face_color='blue', edge_color=None, size=5)
    
    # Clear the canvas
    canvas.update()

# Create the animation
animation = app.Timer(connect=update, start=False)

# Save the animation as a GIF
# animation.save('particle_animation.gif', writer='pillow')  # VisPy doesn't natively support GIF export

# Start the animation (you can stop it with animation.stop())
animation.start()

if __name__ == '__main__':
    app.run()
